# imports

In [ ]:
pip install xlsxwriter


     |████████████████████████████████| 149 kB 3.8 MB/s 


In [ ]:
import zipfile
import pandas as pd
import numpy as np
import matplotlib as plt
import random as rd
import time
import os
import openpyxl
import xlsxwriter
import math



# Main

In [ ]:
writer1= pd.ExcelWriter('Vecindarios_variables.xlsx',engine='xlsxwriter')
writer2= pd.ExcelWriter('Multiples_caracteristicas.xlsx',engine='xlsxwriter')
k=3
s= zipfile.ZipFile('pmedcap (3).zip','r')
Zipfile = ('pmedcap.zip','r')
with s as zipObj:
  listOfiles = zipObj.namelist()
  print(listOfiles)
  for elem in listOfiles:
    txtdata = s.extract(elem)
    listN = []
    with open(txtdata) as f:
      lines = f.readline()
      print(lines)
      list1 = list(lines.split("\t"))
      print(list1)
      lista = list(list1[0:2])
      print(lista)
      results = [int(i) for i in lista]
      while lines:
        lines = f.readline()
        if lines == "":
          break
        else:
          list1 = list(lines.split("\t"))
          lista = [int(i) for i in list1]
          lista = np.array(lista)
          listN.append(lista)

    listNodos = np.array(listN.copy())
    numNodos = results[0]
    numPmedians = results[1]
    vvn = varios_vecindarios(listNodos, numPmedians, numNodos)
    vc= varias_caracteristicas(listNodos,numPmedians,numNodos)
    print(elem)
    vvn.to_excel(writer1,sheet_name=elem)
    vc.to_excel(writer2, sheet_name = elem)

writer1.save()
writer2.save()


['pmedcap11.txt', 'pmedcap12.txt', 'pmedcap13.txt', 'pmedcap14.txt', 'pmedcap15.txt']
3038	1000		

['3038', '1000', '', '\n']
['3038', '1000']
[[400, 41, 43, 45, 47], [401, 49, 50, 52, 53, 60, 219], [402, 51, 416], [403, 56, 63, 81, 259], [404, 62, 64], [405, 79, 82, 241], [406, 137, 138, 254], [407, 139, 237], [408, 239, 242, 244], [409, 243, 246], [410, 77, 85], [411, 75, 76, 84], [412, 70, 71, 72, 73], [413, 66, 68, 80], [414, 65, 69, 74, 245], [415, 54, 55, 78, 83, 261], [432, 86, 87, 98, 421], [433, 88, 93, 274], [434, 90, 92, 103], [435, 89, 91, 102], [580, 8, 15, 24], [581, 20, 21, 35], [582, 26, 28, 29, 40, 374, 598], [583, 34, 36, 37, 214], [584, 193, 194, 204, 207, 593], [585, 39, 42, 213], [591, 22, 23, 25, 48, 379], [603, 44, 46, 57], [604, 209, 211], [605, 217, 222, 224], [606, 223, 386], [607, 231, 233, 590], [608, 378], [609, 383, 384], [610, 393, 394, 395], [611, 418, 428], [612, 420], [613, 426, 429], [614, 430], [615, 422, 425], [616, 419], [617, 398, 399, 417], [618,

In [ ]:
s= zipfile.ZipFile('pmedcap.zip','r')
Zipfile = ('pmedcap.zip','r')
with s as zipObj:
  listOfiles = zipObj.namelist()
  #print(listOfiles)
  for elem in listOfiles:
    txtdata = s.extract(elem)
    listN = []
    with open(txtdata) as f:
      lines = f.readline()
      #print(lines)
      list1 = list(lines.split("\t"))
      #print(list1)
      lista = list(list1[0:2])
      #print(lista)
      results = [int(i) for i in lista]
      while lines:
        lines = f.readline()
        if lines == "":
          break
        else:
          list1 = list(lines.split("\t"))
          lista = [int(i) for i in list1]
          lista = np.array(lista)
          listN.append(lista)

    listNodos = np.array(listN.copy())
    numNodos = results[0]
    numPmedians = results[1]
    vvn = varios_vecindarios(listNodos, numPmedians, numNodos)
    print(vvn)
    vc = varias_caracteristicas(listNodos,numPmedians,numNodos)
    print(vc)

NameError: ignored

# Vecindarios variables

In [ ]:
def varios_vecindarios(listNodos, numPmedians, numNodos):
  #Solución inicial
  start_time = time.time()
  posibles = np.zeros(numNodos)
  listD = matrizDistancias(listNodos, numNodos, numPmedians)
  
  noice = constructivo(listNodos, numPmedians, numNodos, listD)
  posibles[noice] = 1

  anexoInicial, foInicial, listNodosI = calculoFo(posibles, listD, listNodos)
  posiblesI = posibles.copy()
  print(anexoInicial)
  print(foInicial)

  while time.time() - start_time < 60*10:
    posibles, anexoInicial, foInicial, listNodosI = vecindarioEleccionPmeds(listNodos, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD, listNodosI)
    if time.time() -start_time > 60*10:
      break
    posibles, anexoInicial, foInicial, listNodosI = vecindarioIntercambioAsignacion(listNodosI, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD, listNodos)
    if time.time() -start_time > 60*10:
      break
    posibles, anexoInicial, foInicial, listNodosI = vecindarioReasignacion(listNodosI, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD, 1, listNodos)
  posibles1 = np.where(posibles != 0)[0]
  ultimaFila = [foInicial, time.time() - start_time ]
  fos = calculoFo2(posibles, anexoInicial, listD)
  dfout = formato(posibles1, anexoInicial, ultimaFila, numPmedians, fos)
  return dfout


          

# Multiples caracteristicas

In [ ]:
def varias_caracteristicas(listNodos, numPmedians, numNodos):
  #Solución inicial
  start_time = time.time()
  listD = matrizDistancias(listNodos, numNodos, numPmedians)  
  noice = constructivo(listNodos, numPmedians, numNodos, listD)
  pos = np.zeros(numNodos)
  pos[noice] = 1
  anexoInicial1, foInicial1, listNodosI1 = calculoFo(pos, listD, listNodos)
  solIn = variasSolucionesIniciales(numNodos, numPmedians, 5, pos)

  for posibles in solIn:
    startTime1 = time.time()
    anexoInicial, foInicial, listNodosI = calculoFo(posibles, listD, listNodos)
    posiblesI = posibles.copy()
    while time.time() - startTime1 < 100:
      posibles, anexoInicial, foInicial, listNodosI = destruccionConstruccion(listNodosI, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD, 5, 10, listNodos)
      if time.time() -startTime1 > 100:
        break
      posibles, anexoInicial, foInicial, listNodosI = recocidoSimulado(listNodos, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD, listNodosI, T0=150, iter=10, g=0.99)
    if foInicial < foInicial1:
      noice, anexoInicial1, foInicial1, listNodosI1 = posibles, anexoInicial, foInicial, listNodosI
      print(faxCheck(listNodos, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD))
  fos = calculoFo2(noice, anexoInicial1, listD)
  noice1 = np.where(noice != 0)[0]
  ultimaFila = [foInicial1, time.time() - start_time]
  dfout = formato(noice1, anexoInicial1, ultimaFila, numPmedians, fos)
  return dfout

# Generación vecindarios

In [ ]:

def vecindarioEleccionPmeds(listNodos, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD, listNodosI):
  start_time1 = time.time()
  for i in range(len(posibles)):
    if posibles[i] == 1:
      for j in range(len(posibles)):
        if posibles[j] == 0:
          xp = posibles.copy()
          xp[i] = 0
          xp[j] = 1
          anexo, fo, listN = calculoFo(xp, listD, listNodos)
          #mejoria
          if fo < foInicial :
            return xp, anexo, fo, listN
          #Parada de emergencia
          if time.time()-start_time1 >= 40:
            return posibles, anexoInicial, foInicial, listNodosI
  return posibles, anexoInicial, foInicial, listNodosI


In [ ]:
def vecindarioIntercambioAsignacion(listNodos, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD, listN):
  start_time1 = time.time()
  bases = np.where(posibles != 0 )[0]
  for i in range(numPmedians - 1):
    for j in range(i+1, numPmedians):
      for k in range(1, len(anexoInicial[i])):
        for l in range(1, len(anexoInicial[j])):
          aux = listNodos.copy()
          x = anexoInicial.copy()
          temp = x[i][k]
          x[i][k] = x[j][l]
          x[j][l] = temp
          fo = calculoFo1(posibles, x, listD)
          aux[bases[i], 3] -= aux[anexoInicial[i][k], 3]
          aux[bases[i], 3] += aux[anexoInicial[j][l], 3]
          aux[bases[j], 3] -= aux[anexoInicial[j][l], 3]
          aux[bases[j], 3] += aux[anexoInicial[i][k], 3]
          if fo < foInicial and faxCheck(listN, numPmedians, numNodos, posibles, x, foInicial, listD):
            return posibles, x, fo, aux
          if time.time() - start_time1 >= 40:
            return posibles, anexoInicial, foInicial, listNodos
  return posibles, anexoInicial, foInicial, listNodos

In [ ]:
def vecindarioReasignacion(listNodos, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD, n, listNo):
  start_time1 = time.time()

  for l in range(n, n):
    aux = listNodos.copy()
    auxiliar = anexoInicial.copy()
    x = np.where(posibles != 0)[0]
    borrados = []
    for i in range(numPmedians):
      array = anexoInicial[i].copy()
      dists = []
      for j in array:
        dis = listD[x[i], j]
        dists.append(dis)
      sort = sorted(range(len(dists)), key = lambda k: dists[k])
      sort.reverse()
      for j in range(l):
        ind = sort[j]
        node = array[ind]
        aux[x[i], 3] -= aux[node ,3]
        borrados.append(node)
        auxiliar[i].remove(node)
    listN, anexados = ReasignarNodos(aux, numPmedians, numNodos, posibles, auxiliar, foInicial, listD, l, borrados)
    fo = calculoFo1(posibles, anexados, listD)
    if fo < foInicial and faxCheck(listNo, numPmedians, numNodos, posibles, anexados, foInicial, listD):
      return  posibles, anexados, fo, listN
    if time.time() - start_time1 >= 40:
            return posibles, anexoInicial, foInicial, listNodos
  return posibles, anexoInicial, foInicial, listNodos

# Caracteristicas y vecindarios

In [ ]:
def destruccionConstruccion(listNodos, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD, l, m, listNo):
  start_time1 = time.time()
  
  for i in range(l, m):
    aux = listNodos.copy()
    auxiliar = anexoInicial.copy()
    x = np.where(posibles != 0)[0]
    borrados = []
    for j in range(numPmedians):
      elim = int((len(anexoInicial[j])/100) * i)
      indexes = rd.sample(range(1, len(anexoInicial[j])), elim)
      for k in indexes:
        node = anexoInicial[j][k]
        aux[x[j], 3] -= aux[node, 3]
        borrados.append(node)
        auxiliar[j].remove(node)
    listN, anexados = ReasignarNodos(aux, numPmedians, numNodos, posibles, auxiliar, foInicial, listD, l, borrados)
    fo = calculoFo1(posibles, anexados, listD)
    if fo < foInicial and faxCheck(listNo, numPmedians, numNodos, posibles, anexados, foInicial, listD):
      return  posibles, anexados, fo, listN
    if time.time() - start_time1 >= 40:
      return posibles, anexoInicial, foInicial, listNodos
    return posibles, anexoInicial, foInicial, listNodos

In [ ]:
def variasSolucionesIniciales(numNodos, numPmedians, m, noice):
  solIn = [noice]
  for i in range(m):
    indexes = rd.sample(range(numNodos), numPmedians)
    posibles = np.zeros(numNodos)
    for i in indexes:
     posibles[i] = 1
    solIn.append(posibles)
  return solIn

In [ ]:
def recocidoSimulado(listNodos, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD, listNodosI, T0=150, iter=10, g=0.99):
  mejorFo = foInicial
  mejorAnexo = anexoInicial
  mposibles = posibles
  mejorlistNodos = listNodos
  temp = T0
  for i in range(iter):
    posibles, anexoInicial, foInicial, listNodosI = vecindarioEleccionPmeds(listNodos, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD, listNodosI)
    delta = foInicial - mejorFo
    if delta<= 0:
      prob = 1
    else:
      prob = math.exp(-delta/temp)
    r = rd.random()
    if prob >= r:
      mejorFo = foInicial
      mejorAnexo = anexoInicial
      mposibles = posibles
      mejorlistNodos = listNodosI
    temp = temp * g
  return mposibles, mejorAnexo, mejorFo, mejorlistNodos

# Auxiliares

In [ ]:
def faxCheck(listNodos, numPmedians, numNodos, posibles, anexoInicial, foInicial, listD):
  aux = listNodos.copy()
  caps = []
  dems = 0
  for i in range(numPmedians):
    cent = anexoInicial[i][0]
    dem = aux[cent, 3] 
    for j in range(1, len(anexoInicial[i])):
      dem += aux[anexoInicial[i][j], 3]
    dems += dem
    leftCap = aux[cent, 2] - dem
    #print(leftCap)
    caps.append(leftCap)
  boole = True
  for i in caps:
    if i<0:
      boole = False
  return boole

In [ ]:
def ReasignarNodos(listNodos, numPmedians, numNodos, posibles, anexoInicial, foInicial, listDistances, n, borrados):
  aux = listNodos.copy()
  ayuda = anexoInicial.copy()
  ayuda1 = np.where(posibles != 0 )[0]
  borrados.reverse()
  #print(ayuda)
  for j in range(len(borrados)):
    #print(borrados[j])
    array = listDistances[:,borrados[j]]
    sort = sorted(range(len(array)), key = lambda k: array[k])
    if borrados[j] not in ayuda1:
      for v in sort:
            if (aux[v, 2] >= (aux[v, 3] + aux[borrados[j], 3])) and (v in ayuda1):
              #print(borrados[j])
              addTo = np.where(ayuda1 == v)[0][0]
              aux[v, 3] += aux[borrados[j], 3]
              ayuda[addTo].append(borrados[j])
              #print(ayuda)
              break
            #if v in ayuda1:
              #print(aux[v,2] - (aux[v,3] + aux[borrados[j], 3]))
  return aux, ayuda

In [ ]:
def matrizDistancias(listNodos, numNodos, numPmedians):
  aux = listNodos.copy()
  listDistances = np.zeros([numNodos, numNodos])
  for i in range(numNodos):
    listDistance = np.zeros(numNodos)
    for n in range( numNodos):
      distance = distancia(aux[i, 0], aux[n, 0], aux[i, 1], aux[n, 1]) 
      listDistance[n] = distance
    listDistances[i] += listDistance
  return listDistances

In [ ]:
def distancia(a1, a2, b1, b2):
  r = np.sqrt((a1-a2)**2+(b1-b2)**2)
  return r

In [ ]:
def calculoFo(vecino, listDistances, listNodos):
  posibles = np.where(vecino != 0 )[0]
  aux = listNodos.copy()
  ayuda = list(map(lambda el:[el], posibles.copy()))
  ayuda1 = posibles.copy()

  for j in range(numNodos):
    array = listDistances[:,j]
    sort = sorted(range(len(array)), key = lambda k: array[k])
    if j not in ayuda1:
      for v in sort:
            if ((aux[v, 2] >= aux[v, 3] + aux[j, 3]) and (v in ayuda1)):
              addTo = np.where(ayuda1 == v)[0][0]
              aux[v, 3] += aux[j, 3]
              ayuda[addTo].append(j)
              break
  
  
  fo = np.zeros(len(posibles))
  ayuda1 = np.array(posibles.copy())
  for i in posibles:
    ayu = np.where(ayuda1 == i)
    added = 0
    for j in ayuda[ayu[0][0]]:
      added += int(listDistances[i, j])
    fo[ayu[0][0]] += added
  sumaTotal = sum(fo)
  return ayuda, sumaTotal, aux

In [ ]:
def calculoFo1(vecino, anexados, listDistances):
  posibles = np.where(vecino != 0 )[0]
  fo = np.zeros(len(posibles))
  ayuda1 = np.array(posibles.copy())
  for i in posibles:
    ayu = np.where(ayuda1 == i)
    added = 0
    for j in anexados[ayu[0][0]]:
      added += int(listDistances[i, j])
    fo[ayu[0][0]] += added
  sumaTotal = sum(fo)
  return sumaTotal


In [ ]:
def calculoFo2(vecino, anexados, listDistances):
  posibles = np.where(vecino != 0 )[0]
  fo = np.zeros(len(posibles))
  ayuda1 = np.array(posibles.copy())
  for i in posibles:
    ayu = np.where(ayuda1 == i)
    added = 0
    for j in anexados[ayu[0][0]]:
      added += int(listDistances[i, j])
    fo[ayu[0][0]] += added
  return fo

In [ ]:
def constructivo(listNodos, numPmedians, numNodos, listDistances):
  possibles = []
  sumDist = sum(listDistances)
  sortedDistances = sorted(range(len(sumDist)), key = lambda k: sumDist[k])
  for i in range(numPmedians):
    possibles.append(sortedDistances[i])
  possibles.sort()
  return possibles

In [ ]:
def formato(bases, anexos, ultimaFila, numPmedians, fos): 
  cuantosHay = []
  for i in range(len(anexos)):
    anexos[i].append(fos[i])
  for i in anexos:
    cuantosHay.append(len(i) - 1)
  df= np.empty((numPmedians+1,max(cuantosHay)+4))
  df[:]= np.nan
  df= pd.DataFrame(df) #creo el data frame
  for i in range(numPmedians):
    df.loc[i,0]= bases[i] + 1
    df.loc[i,1]= cuantosHay[i]
    for j in range(len(anexos[i])):
      df.loc[i,j+2]= int(anexos[i][j]) + 1
  df.loc[numPmedians,0]= int(ultimaFila[0])
  df.loc[numPmedians,1]= ultimaFila[1] 
  return df